# Agentic AI MCP - Client Only

This notebook connects to a remote MCP server and runs agentic workflows.
The MCP server should be running on another machine (see `quickstart_server.ipynb`).

## Step 1: Setup

In [ ]:
# install if needed
# !uv pip install agentic-ai-mcp==0.5.9

import agentic_ai_mcp

print(f"Version: {agentic_ai_mcp.__version__}")

## Step 2: Make Sure To Have A Valid API_KEY

The LLM runs on this machine, so you need the API key here.
- For **Anthropic** (default): Set `ANTHROPIC_API_KEY`
- For **OpenAI**: Set `OPENAI_API_KEY`

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

# check for API keys
if os.getenv("ANTHROPIC_API_KEY"):
    print("ANTHROPIC_API_KEY is set")
else:
    print("WARNING: ANTHROPIC_API_KEY not found")

if os.getenv("OPENAI_API_KEY"):
    print("OPENAI_API_KEY is set")
else:
    print("INFO: OPENAI_API_KEY not found (optional, only needed for OpenAI provider)")

## Step 3: Connect to MCP Server and Create AgenticAIClient

Use the `mcp_url` parameter to connect to an existing MCP server.

In [3]:
from agentic_ai_mcp import AgenticAIClient

# connect to MCP server (change this to your server's URL)
MCP_SERVER_URL = "http://127.0.0.1:8888/mcp"

# default: provider = 'anthropic', model = 'claude-haiku-4-5-20251001'
client = AgenticAIClient(mcp_url=MCP_SERVER_URL, verbose=True)

# if you want to use OpenAI model, change accordingly, e.g. -
# client = AgenticAIClient(mcp_url=MCP_SERVER_URL, provider="openai", model="gpt-4o-mini")

# you can also pass the API key directly:
# client = AgenticAIClient(mcp_url=MCP_SERVER_URL, api_key="sk-...")

print(f"Client configured to connect to: {MCP_SERVER_URL}")

Client configured to connect to: http://127.0.0.1:8888/mcp


## Step 4a: Run Agent (Simple Task)

In [4]:
result = await client.run("Calculate 2+1, then use the result as the number of times to greet 'Alice'.")
print()
print(">>>")
print()
print(result)

Loading tools from: http://127.0.0.1:8888/mcp
Loaded tools: ['add', 'multiply', 'greet']

PROMPT: Calculate 2+1, then use the result as the number of times to greet 'Alice'.



/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/client.py:135: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self._agent = create_react_agent(


STEP 1: add({'a': 2, 'b': 1})
  → CallToolResult(content=[TextContent(type='text', text='{"result":3}', annotations=None, meta=None)], structured_content={'result': 3}, meta=None, data={'result': 3}, is_error=False)

STEP 2: greet({'name': 'Alice', 'times': 3})
  → CallToolResult(content=[TextContent(type='text', text='{"result":"Hello, Alice! Hello, Alice! Hello, Alice! "}', annotations=None, meta=None)], structured_content={'result': 'Hello, Alice! Hello, Alice! Hello, Alice! '}, meta=None, data={'result': 'Hello, Alice! Hello, Alice! Hello, Alice! '}, is_error=False)

RESULT: Perfect! Here are the results:
- **2 + 1 = 3**
- **Greeting Alice 3 times:** "Hello, Alice! Hello, Alice! Hello, Alice!"


>>>

Perfect! Here are the results:
- **2 + 1 = 3**
- **Greeting Alice 3 times:** "Hello, Alice! Hello, Alice! Hello, Alice!"


## Step 4b: Run Agent (Complex Task with Planning)

In [5]:
result = await client.run_with_planning("Calculate (((0+2) x (1+1)) + ((0+1) x (1+1))), then use the result as the number of times to greet 'Bob'.")
print()
print(">>>")
print()
print(result)


PLANNING MODE
TASK: Calculate (((0+2) x (1+1)) + ((0+1) x (1+1))), then use the result as the number of times to greet 'Bob'.

PLAN:
  1. Use add(a=0, b=2) to calculate the first inner sum
  2. Use add(a=1, b=1) to calculate the second inner sum
  3. Use multiply(a=result from step 1, b=result from step 2) to multiply those results
  4. Use add(a=0, b=1) to calculate the third inner sum
  5. Use multiply(a=result from step 4, b=result from step 2) to multiply those results
  6. Use add(a=result from step 3, b=result from step 5) to calculate the final sum
  7. Use greet(name="Bob", times=result from step 6) to greet Bob

EXECUTING STEP 1/7: Use add(a=0, b=2) to calculate the first inner sum
----------------------------------------


/mnt/jawad_cnms_s1/jawad_server/code/20260221_code_agentic_ai/agentic-ai-mcp/src/agentic_ai_mcp/workflows/planning.py:151: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  step_agent = create_react_agent(llm, tools)


  TOOL CALL 1: add({'a': 0, 'b': 2})
    → CallToolResult(content=[TextContent(type='text', text='{"result":2}', annotations=None, meta=None)], structured_content={'result': 2}, meta=None, data={'result': 2}, is_error=False)
  AI RESPONSE: The first inner sum has been calculated. Using `add(a=0, b=2)`, the result is **2**.
----------------------------------------
STEP 1 COMPLETE

EXECUTING STEP 2/7: Use add(a=1, b=1) to calculate the second inner sum
----------------------------------------
  TOOL CALL 1: add({'a': 1, 'b': 1})
    → CallToolResult(content=[TextContent(type='text', text='{"result":2}', annotations=None, meta=None)], structured_content={'result': 2}, meta=None, data={'result': 2}, is_error=False)
  AI RESPONSE: Perfect! Using `add(a=1, b=1)`, the result of the second inner sum is **2**.

**Summary so far:**
- Step 1: First inner sum = 2 (from 0 + 2)
- Step 2: Second inner sum = 2 (from 1 + 1)
----------------------------------------
STEP 2 COMPLETE

EXECUTING STEP 3/7: U